In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [19]:
# Load data
df = pd.read_csv("Global_Education.csv", encoding="latin1")
df.head()

,Countries and areas,Latitude,Longitude,OOSR_Pre0Primary_Age_Male,OOSR_Pre0Primary_Age_Female,OOSR_Primary_Age_Male,OOSR_Primary_Age_Female,OOSR_Lower_Secondary_Age_Male,OOSR_Lower_Secondary_Age_Female,OOSR_Upper_Secondary_Age_Male,...,Primary_End_Proficiency_Reading,Primary_End_Proficiency_Math,Lower_Secondary_End_Proficiency_Reading,Lower_Secondary_End_Proficiency_Math,Youth_15_24_Literacy_Rate_Male,Youth_15_24_Literacy_Rate_Female,Birth_Rate,Gross_Primary_Education_Enrollment,Gross_Tertiary_Education_Enrollment,Unemployment_Rate
0,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,44,...,13,11,0,0,74,56,32.49,104.0,9.7,11.12
1,Albania,41.153332,20.168331,4,2,6,3,6,1,21,...,0,0,48,58,99,100,11.78,107.0,55.0,12.33
2,Algeria,28.033886,1.659626,0,0,0,0,0,0,0,...,0,0,21,19,98,97,24.28,109.9,51.4,11.70
3,Andorra,42.506285,1.521801,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7.20,106.4,0.0,0.00
4,Angola,11.202692,17.873887,31,39,0,0,0,0,0,...,0,0,0,0,0,0,40.73,113.5,9.3,6.89


In [20]:
# Check basic info
print(f"Total records: {df.shape[0]}")
print(f"\nAll countries in dataset:")
print(df['Country'].unique())

Total records: 202

All countries in dataset:


KeyError: 'Country'

In [ ]:
import pandas as pd

df = pd.read_csv("Global_Education.csv", encoding="latin1")

print(f"Shape: {df.shape}")
print(f"\nAll Columns:")
for col in df.columns:
    print(f"  - {col}")
print(f"\nData Types:\n{df.dtypes}")

Shape: (202, 29)

All Columns:
  - Countries and areas
  - Latitude 
  - Longitude
  - OOSR_Pre0Primary_Age_Male
  - OOSR_Pre0Primary_Age_Female
  - OOSR_Primary_Age_Male
  - OOSR_Primary_Age_Female
  - OOSR_Lower_Secondary_Age_Male
  - OOSR_Lower_Secondary_Age_Female
  - OOSR_Upper_Secondary_Age_Male
  - OOSR_Upper_Secondary_Age_Female
  - Completion_Rate_Primary_Male
  - Completion_Rate_Primary_Female
  - Completion_Rate_Lower_Secondary_Male
  - Completion_Rate_Lower_Secondary_Female
  - Completion_Rate_Upper_Secondary_Male
  - Completion_Rate_Upper_Secondary_Female
  - Grade_2_3_Proficiency_Reading
  - Grade_2_3_Proficiency_Math
  - Primary_End_Proficiency_Reading
  - Primary_End_Proficiency_Math
  - Lower_Secondary_End_Proficiency_Reading
  - Lower_Secondary_End_Proficiency_Math
  - Youth_15_24_Literacy_Rate_Male
  - Youth_15_24_Literacy_Rate_Female
  - Birth_Rate
  - Gross_Primary_Education_Enrollment
  - Gross_Tertiary_Education_Enrollment
  - Unemployment_Rate

Data Types:
Count

In [ ]:
# List of African countries
african_countries = [
    'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi',
    'Cabo Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros',
    'Congo', "Côte d'Ivoire", 'Djibouti', 'Egypt', 'Equatorial Guinea',
    'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea',
    'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar',
    'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique',
    'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe',
    'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa',
    'South Sudan', 'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Uganda',
    'Zambia', 'Zimbabwe', 'Democratic Republic of the Congo',
    'United Republic of Tanzania', 'Republic of the Congo', 'Ivory Coast',
    "Cote d'Ivoire", 'Cape Verde', 'Swaziland'
]

# Filter for African countries
df_africa = df[df['Countries and areas'].isin(african_countries)]

print(f"African countries found: {df_africa.shape[0]}")
print(f"\nCountries in dataset:")
print(df_africa['Countries and areas'].tolist())

African countries found: 51

Countries in dataset:
['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cape Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros', 'Republic of the Congo', 'Ivory Coast', 'Democratic Republic of the Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon', 'Ghana', 'Guinea', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Togo', 'Tunisia', 'Uganda', 'Tanzania', 'Zambia', 'Zimbabwe']


In [ ]:
african_countries = [
    'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi',
    'Cape Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros',
    'Congo', 'Republic of the Congo', 'Ivory Coast', 'Democratic Republic of the Congo',
    'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia',
    'Gabon', 'Ghana', 'Guinea', 'Kenya', 'Lesotho', 'Liberia', 'Libya',
    'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco',
    'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Senegal',
    'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan',
    'Sudan', 'Togo', 'Tunisia', 'Uganda', 'Tanzania', 'Zambia', 'Zimbabwe'
]

df_africa = df[df['Countries and areas'].isin(african_countries)]

print(f"Shape: {df_africa.shape}")
print(f"\nMissing values:\n{df_africa.isnull().sum()}")
print(f"\nBasic stats for potential target variables:")
print(df_africa[['Completion_Rate_Upper_Secondary_Male', 'Completion_Rate_Upper_Secondary_Female', 
                  'Unemployment_Rate', 'Youth_15_24_Literacy_Rate_Male']].describe())


Shape: (51, 29)

Missing values:
Countries and areas                        0
Latitude                                   0
Longitude                                  0
OOSR_Pre0Primary_Age_Male                  0
OOSR_Pre0Primary_Age_Female                0
OOSR_Primary_Age_Male                      0
OOSR_Primary_Age_Female                    0
OOSR_Lower_Secondary_Age_Male              0
OOSR_Lower_Secondary_Age_Female            0
OOSR_Upper_Secondary_Age_Male              0
OOSR_Upper_Secondary_Age_Female            0
Completion_Rate_Primary_Male               0
Completion_Rate_Primary_Female             0
Completion_Rate_Lower_Secondary_Male       0
Completion_Rate_Lower_Secondary_Female     0
Completion_Rate_Upper_Secondary_Male       0
Completion_Rate_Upper_Secondary_Female     0
Grade_2_3_Proficiency_Reading              0
Grade_2_3_Proficiency_Math                 0
Primary_End_Proficiency_Reading            0
Primary_End_Proficiency_Math               0
Lower_Secondary_End_Pr

In [ ]:
# Drop non-numeric columns for correlation
df_numeric = df_africa.drop(['Countries and areas', 'Latitude ', 'Longitude'], axis=1)

# FIGURE 1: Correlation Heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(df_numeric.corr(), annot=False, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap - Education Indicators in Africa')
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=300)
plt.show()

NameError: name 'plt' is not defined